### Extracts energy data from across campus for 2017-2019. 

In [ ]:
import ast
import calendar
from datetime import datetime, timedelta
from dateutil import relativedelta
import gc
import json
import numpy as np
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth

## The code threw up some SettingWithCopyWarnings that I will fix at somepoint
import warnings
warnings.filterwarnings('ignore')

secrets = json.load(open('../secrets.json'))

In [ ]:
# Define headers for API calls
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-GB,en;q=0.5',
    'Content-Type': 'application/json; charset=utf-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://metering.dev.urbanobservatory.ac.uk',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Referer': 'https://metering.dev.urbanobservatory.ac.uk/utilities/currentmetervalues.aspx',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
    'TE': 'Trailers',
}

# Define authorisation credentials. 
# NOTE! These are for the UO proxy of the metering.ncl.ac.uk website. 
# They will not work for the main website, and other authorisation may be required.
auth = (
    secrets['metering']['username'],
    secrets['metering']['password']
)

In [ ]:
# Call metadata
url = 'https://metering.dev.urbanobservatory.ac.uk/utilities/currentmetervalues.aspx/GetMeters'

r = requests.post(url=url, headers=headers,auth=auth)

In [ ]:
# Extract JSON data, import to dataframe. 
sensorData = r.json()
sensorData = sensorData["d"]
sensorData = json.loads(sensorData)

cols= list(sensorData[0].keys())
sensorDF = pd.DataFrame(columns = cols)

for s in range(1,len(sensorData)):
    row = sensorData[s]
    sensorDF = sensorDF.append(row, ignore_index=True)

In [ ]:
# Hardcoded list of selected sensors - to be updated for more dynamic selection of locations
sensorID = ['CLZ_E_Tx1',    # Castle Leazes 
           'DEV_E_Main',   # Devonshire 
           'HAN_E_Main',   # Great North Museum 
           'REF_E_Main',   # Kings Road Centre
           'PCY_E_Main',   # Percy Building
           'CSH_E_Main',   # Sports Center 
           'UNS_E_Main',   # Students Union
           'W22_E_Main',   # Windsor Terrace (Law)
           'WOL_E_Main']   # Wolfson Building

In [ ]:
# Function to call data from METERology
def callData(location, startDate, endDate, utility):
    print(datetime.now().strftime('%H:%M:%S'))
    
    # Determine number months in date range.
    # If less than 10, run API call, if greater than 10, run API calls
    monthList = pd.date_range(startDate,endDate,freq='MS').strftime("%Y-%m").tolist()
    monthNum = len(monthList)
    
    paramList = list()
    if monthNum <=15:
        dataSplit = False
        for sid in location:
            params = {"meterID":sid,
                    "startDateISO":startDate,
                    "endDateISO":endDate}
            params = str(params)
            paramList.append(params)
    else:
        dataSplit = True
        numChunks = int(len(monthList)/15) + (len(monthList)%15 > 0)
        for sid in location:
            iter = 1
            sd = 0
            ed = 14
            while iter <= numChunks:
                start = monthList[sd]+'-01'
                if iter == numChunks:
                    end = monthList[-1] + '-' + str(calendar.monthrange(int(monthList[-1][:4]), int(monthList[-1][-2:]))[1])
                else:
                    end = monthList[ed] + '-' + str(calendar.monthrange(int(monthList[ed][:4]), int(monthList[ed][-2:]))[1])
                params = {"meterID":sid,
                        "startDateISO":start,
                        "endDateISO":end}
                params = str(params)
                paramList.append(params)
                sd = sd + 15
                ed = ed + 15
                iter = iter+1
        
    # Update params for request session
    with requests.Session() as s:
        s.auth = auth
        s.headers.update({
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'en-GB,en;q=0.5',
            'Content-Type': 'application/json; charset=utf-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Origin': 'https://metering.dev.urbanobservatory.ac.uk',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Referer': 'https://metering.dev.urbanobservatory.ac.uk/utilities/historicalhalfhourreadings.aspx',
            'Pragma': 'no-cache',
            'Cache-Control': 'no-cache'
        })

        url = "https://metering.dev.urbanobservatory.ac.uk/utilities/historicalhalfhourreadings.aspx/GetReadings"
        
        resultsDict = dict()
        
        current = 1
        total = len(paramList)
        
        for par in paramList:
            sid = ast.literal_eval(par)
            sid = sid.get('meterID')
            
            if dataSplit == True:
                sid = sid + '---Iter' + str(current)
            
            print(sid,":","(",current,"/",total,")")
        
            r = s.post(url=url,data=par)
            
            data = r.json()
            data = data["d"]
            data = json.loads(data)
            
            resultsDict.update({sid : data})
        
            current = current + 1
            
    print(datetime.now().strftime('%H:%M:%S'))
    return resultsDict

In [ ]:
# Define parameters, call function using hardcoded sensor list
startDate = "2017-01-01 00:00:00" # Please use YYYY-MM-DD format, or type default. 
endDate = "2020-02-29 23:59:59" # Please use YYYY-MM-DD format, or type default. 
utility = "All" # Options are Electricity, Gas, Heat or Water. Type "All" to download all datasets.

resultsDict = callData(sensorID, startDate, endDate, utility)

In [ ]:
# Reformat Data into DataFrame
reformatDF = pd.DataFrame()

total = len(resultsDict)
current = 1

for sid in resultsDict:
    print(sid,":","(",current,"/",total,")")
    print(datetime.now().strftime('%H:%M:%S'))
    data = resultsDict.get(sid)
    tempDF = pd.DataFrame()
    tempDF = pd.DataFrame(data)
    idString = sid.split("---")
    tempDF["id"] = idString[0]
    reformatDF = reformatDF.append(tempDF, ignore_index = True)
    tempDF = None
    current = current +1 
    
reformatDF = reformatDF.dropna(subset=['reading'])
reformatDF.loc[:,"DateTime"] = reformatDF['date'] + ' ' + reformatDF['time']
reformatDF.loc[:,"DateTime"] = pd.to_datetime(reformatDF['DateTime'],format='%d/%m/%Y %H:%M')
reformatDF.drop(columns=['date','time'], axis=1, inplace=True)

# Create List of DateTimes of Expected Readings (Every 30 mins)
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
        
first = "01/01/2017 00:00" 
last = "29/02/2020 23:45" 
        
expectedDates = [dt.strftime('%d/%m/%Y %H:%M') for dt in 
       datetime_range(datetime.strptime(first, '%d/%m/%Y %H:%M'), datetime.strptime(last, '%d/%m/%Y %H:%M'), 
       timedelta(minutes=30))]

# Make DataFrame user-friendly
resultsDF = pd.DataFrame(index=expectedDates)

for sid in sensorID:
    tempDF = reformatDF[reformatDF['id']==sid]
    tempDF.index = tempDF["DateTime"]
    tempDF = tempDF.drop(columns=['id','DateTime'], axis=1)
    
    resultsDF = resultsDF.merge(tempDF, how='outer',left_index=True, right_index=True)
    resultsDF.columns = [*resultsDF.columns[:-1], sid]
    
resultsDF = resultsDF.dropna(how='all')

In [ ]:
pd.to_pickle(sensorDF, '../cache/energy-metadata.pkl')
pd.to_pickle(resultsDF, '../cache/energy-baseline.pkl')
gc.collect()